In [1]:
import pandas as pd

## Data Processing 

In [2]:
# Load the ratings data
ratings = pd.read_csv('ml-100k/u.data', sep='\t', header=None, names=['user_id', 'item_id', 'rating', 'timestamp'])

# Load the movies data
movies = pd.read_csv('ml-100k/u.item', sep='|', encoding='latin-1', header=None,
                     names=['item_id', 'title'] + [f'genre_{i}' for i in range(19)])

# Load the user data
users = pd.read_csv('ml-100k/u.user', sep='|', header=None, 
                    names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

In [3]:
users

,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [4]:
user_interactions = ratings.groupby('user_id').size()
user_indexes = user_interactions[user_interactions > 200].index 
ratings = ratings[ratings.user_id.isin(user_indexes)]
ratings

,user_id,item_id,rating,timestamp
3,244,51,2,880606923
8,305,451,3,886324817
9,6,86,3,883603013
10,62,257,2,879372434
11,286,1014,5,879781125
...,...,...,...,...
99994,378,78,3,880056976
99995,880,476,3,880175444
99996,716,204,5,879795543
99997,276,1090,1,874795795


In [5]:
movies = movies[movies.index.get_level_values(0).isin(ratings.item_id)]
movies = movies.dropna(axis=1)
movies

,,,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
1,Toy Story (1995),01-Jan-1995,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,"Sunchaser, The (1996)",25-Oct-1996,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1676,"War at Home, The (1996)",01-Jan-1996,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1677,Sweet Nothing (1995),20-Sep-1996,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1681,You So Crazy (1994),01-Jan-1994,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


## Results

In [6]:
from sklearn.model_selection import train_test_split
import random 

test_user = random.choice(ratings.user_id.unique())

def get_data_for_user(ratings, movies, user_id):    
    temp_ratings = ratings[ratings.user_id == user_id]
    temp_movies = movies[movies.index.get_level_values(0).isin(temp_ratings.item_id)]
    temp_data = temp_movies.merge(temp_ratings, left_on=temp_movies.index.get_level_values(0), right_on='item_id').drop(['item_id', 'user_id', 'timestamp'], axis=1) 
    X = temp_data.drop('rating', axis=1)
    y = temp_data['rating']
    return train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

X_train, X_test, y_train, y_test = get_data_for_user(ratings, movies, test_user)

param_grid = {
    'LogisticRegression': {
        'model': LogisticRegression(),
        'params': {
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'lbfgs']
        }
    },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'params': {
            'n_estimators': [10, 50, 100],
            'max_depth': [None, 10, 20]
        }
    },
    'DecisionTree': {
        'model': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 10, 20],
            'min_samples_split': [2, 5, 10]
        }
    },
    'MLPClassifier': {
        'model': MLPClassifier(max_iter=1000, random_state=42),
        'params': {
            'hidden_layer_sizes': [(100,), (100, 50), (150,), (150, 75)],
            'activation': ['relu'],
            'solver': ['adam'],
            'alpha': [0.0001, 0.001, 0.01]
        }
    }
}

# Perform grid search for each model
best_models = {}
for model_name, model_info in param_grid.items():
    grid_search = GridSearchCV(model_info['model'], model_info['params'], cv=5, scoring='accuracy')
    grid_search.fit(X_train, y_train)
    best_models[model_name] = grid_search.best_estimator_

best_models

In [ ]:
results = {}

for model_name, model in best_models.items():
    results[model_name] = []

for user in ratings.user_id.unique():
    X_train, X_test, y_train, y_test = get_data_for_user(ratings, movies, user)

    for model_name, model in best_models.items():
        model.fit(X_train, y_train)
        model.score(X_test, y_test)
        results[model_name].append(model.score(X_test, y_test))


In [ ]:
results

In [ ]:
results_df = pd.DataFrame(results)
results_df.mean()